In [1]:
import requests
from fake_useragent import UserAgent
from bs4 import BeautifulSoup
from datetime import datetime
from re import search
import json
from time import sleep
from random import uniform

# Парсинг ссылок
Вдохновившись новостями по нобелевской неделе, для парсинга я выбрал сайт Naked Science, который содержит научнопопулярные статьи и новости из мира науки. Из раздела со всеми новостями будем парсить по страницам название каждой статьи, автора, аннотацию, индекс важности (оценка автором важности исследования/открытия для науки), дату публикации, хэштеги, количество просмотров в качестве целевой переменной, и наконец ссылки.

In [2]:
# Так как дата публикации и число просмотров имеет разные форматы, сделаем функцию для их извлечения

def parse_date(date, form='%d.%m.%Y'):
    mapper = {
        'января': '01',
        'февраля': '02',
        'марта': '03',
        'апреля': '04',
        'мая': '05',
        'июня': '06',
        'июля': '07',
        'августа': '08',
        'сентября': '09',
        'октября': '10',
        'ноября': '11',
        'декабря': '12'
    }
    if search(r'\d{1,2}\.\d{2}\.\d{4}', date):
        return date
    else:
        date = date.split()
        day = '0' + date[0] if len(date[0]) < 2 else date[0]
        return '.'.join((day, mapper[date[1]], '2025'))

def parse_views(v):
    if v[-3:] == 'тыс':
        num = v.split()[0]
        num = num.replace(',', '.')
        v = float(num) * (10 ** 3)
    
    return int(v)

In [3]:
def get_info(obj):
    header = obj.find('a', attrs={'class': 'animate-custom'})
    link = header.attrs['href'].strip()
    
    title = header.get_text().split()
    imp_ind = float(title[-1])
    title = ' '.join(title[:-1])

    author = obj.find(lambda tag: tag.name == 'div' and tag.get('class') == ['meta-item', 'meta-item_author'])
    author = author.string.strip()

    date = obj.find('span', attrs={'class': 'echo_date'}).string
    date = parse_date(date.split(', ')[0])

    views = obj.find('span', attrs={'class': 'fvc-count'}).string
    views = parse_views(views)

    annot = obj.find('p').string.strip()

    tags = obj.find(lambda tag: tag.name == 'div' and tag.get('class') == ['terms-items', 'grid'])
    tags = tags.find_all('div', attrs={'class': 'terms-item'})
    tags = [t.get_text().replace('#', '').strip() for t in tags]

    return {
        'title': title,
        'author': author,
        'date': date,
        'imp_ind': imp_ind,
        'views': views,
        'annot': annot,
        'tags': tags,
        'link': link
    }

In [7]:
user_ag = UserAgent()
main_link = 'https://naked-science.ru/article/page/'
attemps = 0
success = []

start, pages = 10, 500
articles, links = [], []

for i in range(start, pages + start): # начнем пасрить с 10 страницы, чтобы количество просмотров и коммментов уже немного установилось
    if i % 5 == 0 and i != start:
        print(f'processing page {i - start}/{pages}')
        with open('articles_links.json', 'w', encoding='UTF-8') as f:
            json.dump(articles, f, ensure_ascii=False, indent=4)
    
    for j in range(3): # Пытаемся спарсить страницу три раза, если не удалось, увеличиваем счеткчик неудачных попыток
        sleep(uniform(1.1, 1.9))
        response = requests.get(main_link + f'{i}/', headers={'User-Agent': user_ag.random})
        if not response.ok:
            print(f'\n  WARNING: page {i} parsing failed\n')
            continue
        else:
            success.append(i)
            attemps = 0
            soup = BeautifulSoup(response.content, 'html.parser')
            news = soup.find('div', attrs={'class': 'news-items'})
            news = news.find_all(lambda tag: tag.name == 'div' and tag.get('class') == ['news-item-left', 'with-bookmark'])
            for x in news:
                try:
                    a = get_info(x)
                    if a['link'] not in links: # Некоторые статьи могут парситься дважды, поэтому проверяем
                        articles.append(a)
                        links.append(a['link'])
                except:
                    pass
            break
    else:
        attemps += 1

    if attemps > 3:
        # Если не удалось спарсить больше трех страниц подряд, будем прерывать процесс
        print(f'\n  WARNING: Parsing was stopped\n')
        break


print(f'\n --- {len(success)}/{pages} pages have been successfully parsed ---')

processing page 5/500
processing page 10/500
processing page 15/500
processing page 20/500
processing page 25/500
processing page 30/500
processing page 35/500
processing page 40/500
processing page 45/500
processing page 50/500
processing page 55/500
processing page 60/500
processing page 65/500
processing page 70/500
processing page 75/500
processing page 80/500
processing page 85/500
processing page 90/500
processing page 95/500
processing page 100/500
processing page 105/500
processing page 110/500
processing page 115/500
processing page 120/500
processing page 125/500
processing page 130/500
processing page 135/500
processing page 140/500
processing page 145/500
processing page 150/500
processing page 155/500
processing page 160/500
processing page 165/500
processing page 170/500
processing page 175/500
processing page 180/500
processing page 185/500
processing page 190/500
processing page 195/500
processing page 200/500
processing page 205/500
processing page 210/500
processing p

# Парсинг статей

In [8]:
with open('articles_links.json', 'r', encoding='UTF-8') as f:
    articles = json.load(f)
len(articles)

8272

In [37]:
def parse_page(responce, par_sep='\n'):
    soup = BeautifulSoup(response.content, 'html.parser')
    obj = soup.find('div', attrs={'class': 'body'}).find_all('p')
    text = par_sep.join([par.get_text().strip() for par in obj])

    comments = soup.find('div', attrs={'class': 'shesht-comments-list'})
    comments = comments.find_all('div', attrs={'class': 'shesht-comment-template__content-text'})
    comments = [co.get_text().strip() for co in comments]

    return {'text': text, 'comments': comments}

In [ ]:
user_ag = UserAgent()
attemps = 0
success = []

all_pages = len(articles)
pages, links = [], [x['link'] for x in articles]

for i, link in enumerate(links):
    if i % 5 == 0:
        print(f'processing page {i}/{all_pages}')
        with open('texts_comments.json', 'w', encoding='UTF-8') as f:
            json.dump(pages, f, ensure_ascii=False, indent=4)
    
    for j in range(3): # Пытаемся спарсить страницу три раза, если не удалось, увеличиваем счеткчик неудачных попыток
        sleep(uniform(1.1, 1.9))
        response = requests.get(link, headers={'User-Agent': user_ag.random})
        if not response.ok:
            print(f'\n  WARNING: page {i} parsing failed\n')
            continue
        else:
            success.append(i)
            attemps = 0
            try:
                a = parse_page(response)
                a['link'] = link
                pages.append(a)
            except:
                pass
            break
    else:
        attemps += 1

    if attemps > 3:
        # Если не удалось спарсить больше трех страниц подряд, будем прерывать процесс
        print(f'\n  WARNING: Parsing was stopped\n')
        break


print(f'\n --- {len(success)}/{pages} pages have been successfully parsed ---')